In [ ]:
from ucl.common import byte_print, decode_version, decode_sn, getVoltage, pretty_print_obj, lib_version
from ucl.lowState import lowState
from ucl.lowCmd import lowCmd
from ucl.unitreeConnection import unitreeConnection, LOW_WIFI_DEFAULTS, LOW_WIRED_DEFAULTS
from ucl.enums import GaitType, SpeedLevel, MotorModeLow
from ucl.complex import motorCmd, motorCmdArray
import time
import sys
import math
import numpy as np
from pprint import pprint


def jointLinearInterpolation(initPos, targetPos, rate):

    rate = np.fmin(np.fmax(rate, 0.0), 1.0)
    p = initPos*(1-rate) + targetPos*rate
    return p

# You can use one of the 3 Presets WIFI_DEFAULTS, LOW_CMD_DEFAULTS or HIGH_CMD_DEFAULTS.
# IF NONE OF THEM ARE WORKING YOU CAN DEFINE A CUSTOM ONE LIKE THIS:
#
# MY_CONNECTION_SETTINGS = (listenPort, addr_wifi, sendPort_high, local_ip_wifi)
# conn = unitreeConnection(MY_CONNECTION_SETTINGS)
d = {'FR_0':0, 'FR_1':1, 'FR_2':2,
     'FL_0':3, 'FL_1':4, 'FL_2':5,
     'RR_0':6, 'RR_1':7, 'RR_2':8,
     'RL_0':9, 'RL_1':10, 'RL_2':11 }



In [ ]:
Data 받아서 처리하는 함수 정의
def printRobotStatus(conn,lstate):
    data = conn.getData()
    for paket in data:
        lstate.parseData(paket)
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
        print(f'SN [{byte_print(lstate.SN)}]:\t{decode_sn(lstate.SN)}')
        print(f'Ver [{byte_print(lstate.version)}]:\t{decode_version(lstate.version)}')
        print(f'SOC:\t\t\t{lstate.bms.SOC} %')
        print(f'Overall Voltage:\t{getVoltage(lstate.bms.cell_vol)} mv') #something is still wrong here ?!
        print(f'Current:\t\t{lstate.bms.current} mA')
        print(f'Cycles:\t\t\t{lstate.bms.cycle}')
        print(f'Temps BQ:\t\t{lstate.bms.BQ_NTC[0]} °C, {lstate.bms.BQ_NTC[1]}°C')
        print(f'Temps MCU:\t\t{lstate.bms.MCU_NTC[0]} °C, {lstate.bms.MCU_NTC[1]}°C')
        print(f'FootForce:\t\t{lstate.footForce}')
        print(f'FootForceEst:\t\t{lstate.footForceEst}')
        print(f'IMU Temp:\t\t{lstate.imu.temperature}')
        print(f'MotorState FL_2 MODE:\t\t{lstate.motorState[d["FL_2"]].mode}')
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')

In [2]:
## Initialization ##
print(f'Running lib version: {lib_version()}')
conn = unitreeConnection(LOW_WIFI_DEFAULTS)
conn.startRecv()
lcmd = lowCmd()
# lcmd.encrypt = True
lstate = lowState()
mCmdArr = motorCmdArray()
# Send empty command to tell the dog the receive port and initialize the connection
cmd_bytes = lcmd.buildCmd(debug=False)
conn.send(cmd_bytes)
printRobotStatus(conn,lstate)

Running lib version: 0.2


In [36]:
# define a function to print log data
def printLog(conn,lstate):
    data = conn.getData()
    try:
        paket = data[-1]  # Try to process only the latest packet
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
        lstate.parseData(paket)
        print(f'SN [{byte_print(lstate.SN)}]:\t{decode_sn(lstate.SN)}')
        print(f'Ver [{byte_print(lstate.version)}]:\t{decode_version(lstate.version)}')
        print(f'SOC:\t\t\t{lstate.bms.SOC} %')
        print(f'Overall Voltage:\t{getVoltage(lstate.bms.cell_vol)} mv')  # something is still wrong here ?!
        print(f'Current:\t\t{lstate.bms.current} mA')
        print(f'Cycles:\t\t\t{lstate.bms.cycle}')
        print(f'Temps BQ:\t\t{lstate.bms.BQ_NTC[0]} °C, {lstate.bms.BQ_NTC[1]}°C')
        print(f'Temps MCU:\t\t{lstate.bms.MCU_NTC[0]} °C, {lstate.bms.MCU_NTC[1]}°C')
        print(f'FootForce:\t\t{lstate.footForce}')
        print(f'FootForceEst:\t\t{lstate.footForceEst}')
        print(f'IMU Temp:\t\t{lstate.imu.temperature}')
        print(f'MotorState FR_0 MODE:\t\t{lstate.motorState[d["FR_0"]].mode}')
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")

In [37]:
printLog(conn,lstate)

+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
SN [0402020419020d00]:	('Go1_PRO', '2-4-25[2]')
Ver [000109000130fa7f]:	('0.1.9', '0.1.48')
SOC:			23 %
Overall Voltage:	21120 mv
Current:		-1734 mA
Cycles:			79
Temps BQ:		34 °C, 33°C
Temps MCU:		33 °C, 38°C
FootForce:		[898, 0, 0, 30119]
FootForceEst:		[0, 20821, 0, 23208]
IMU Temp:		79
MotorState FR_0 MODE:		0
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=


In [35]:
lstate.motorState[d["FR_0"]].q

-0.2699544429779053

In [ ]:
# 12개 관절의 각도(q)를 radian 단위의 리스트로 리턴하는 함수
def get_joint_angles_rad(conn,lstate):
    """
    conn에서 최신 패킷을 받아 12개 관절의 각도(q, radian) 리스트를 반환

    Returns:
        q_list (list): 12개 관절의 각도 리스트 (radian, joint 순서는 global 'joint' 리스트와 동일)
                       오류시 None 반환
    """
    data = conn.getData()
    try:
        paket = data[-1]  # 최신 패킷만 처리
        lstate.parseData(paket)
        # 관절 이름이 'joint' 리스트에 있다고 가정
        q_list = [lstate.motorState[d[jname]].q for jname in joint]
        return q_list
    except Exception as e:
        print(f"관절 각도 읽기 오류: {e}")
        return None

In [310]:
## Main Loop ##
Tpi = 0
motiontime = 0
speed = 0.0

while True:
    time.sleep(0.002)
    motiontime += 1
    # if motiontime % 100 == 0: #Print every 100 cycles
    #     printRobotStatus(conn,lstate)

    if( motiontime >= 500): # do action after 1sec | motiontime(+1 by 0.002s) * 500 = 1s
        speed = 2 * math.sin(3*math.pi*Tpi/2000.0)
        mCmdArr.setMotorCmd('FL_2',  motorCmd(mode=MotorModeLow.Servo, q=0, dq = speed, Kp = 0, Kd = 4, tau = 0.0))
        lcmd.motorCmd = mCmdArr
        Tpi += 1

    cmd_bytes = lcmd.buildCmd(debug=False)
    conn.send(cmd_bytes)


KeyboardInterrupt: 

In [139]:
def d2r(deg):
    return math.radians(deg)
def r2d(rad):
    return math.degrees(rad)



In [79]:
mCmdArr.setMotorCmd('FL_2',  motorCmd(mode=MotorModeLow.Servo, q=0.0, dq = 0.1, Kp = 5, Kd = 1, tau = 0.0))
lcmd.motorCmd = mCmdArr
cmd_bytes = lcmd.buildCmd(debug=False)
conn.send(cmd_bytes)

In [213]:
def moveFL2(pos = 0,vel = 0,torque = 0.0) :
    mCmdArr.setMotorCmd('FL_2',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 2, Kd = 0.5, tau = torque))
    lcmd.motorCmd = mCmdArr
    cmd_bytes = lcmd.buildCmd(debug=False)
    conn.send(cmd_bytes)
    
segment = 0
flag = True
while True:
    if flag == True :
        segment += 1
        if segment >= 800 :
            flag = False
            segment = 0
        # moveFL2(d2r(-240), 0.1) # Backward Limit
        moveFL2(0, 1) # Backward Limit
        print("Moving FL_2 Forward")
        time.sleep(0.001)
    if flag == False :
        segment += 1
        if segment >= 500 :
            flag = True
            segment = 0
        moveFL2(0, -1) # Backward Limit
        print("Moving FL_2 Backward")
        time.sleep(0.001)

Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward
Moving FL_2 Forward


KeyboardInterrupt: 

In [312]:
def moveFL2(pos = 0,vel = 0,torque = 0.0) :
    mCmdArr.setMotorCmd('FL_0',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 40, Kd = 1, tau = torque))

    lcmd.motorCmd = mCmdArr
    cmd_bytes = lcmd.buildCmd(debug=False)
    conn.send(cmd_bytes)
    
segment = 0
flag = True
while True:
    if flag == True :
        segment += 1
        if segment >= 500 :
            flag = False
            segment = 0
        # moveFL2(d2r(-240), 0.1) # Backward Limit
        moveFL2(d2r(-30), 0.05) # Backward Limit
        time.sleep(1/50) #50Hz
    if flag == False :
        segment += 1
        if segment >= 500 :
            flag = True
            segment = 0
        moveFL2(d2r(-240), 0.05) # Backward Limit
        time.sleep(1/50) #50Hz

TimeoutError: timed out

In [5]:
math.degrees(-0.9)

-51.56620156177409

In [126]:
for paket in data:
    lstate.parseData(paket)

In [7]:
## 관절 각도 초기값 테스트용 ##
def move(pos = 0,vel = 0,torque = 0.0) :
    mCmdArr.setMotorCmd('FR_1',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 1, Kd = 1, tau = torque))
    mCmdArr.setMotorCmd('FL_1',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 1, Kd = 1, tau = torque))

    lcmd.motorCmd = mCmdArr
    cmd_bytes = lcmd.buildCmd(debug=False)
    conn.send(cmd_bytes)
    

flag = True
count = 0    
while True:
    move(math.radians(-55),0.3)
    time.sleep(1/60)
    

KeyboardInterrupt: 

In [360]:
## 관절 각도 제한값 테스트용 ##
def move(pos = 0,vel = 0,torque = 0.0) :
    mCmdArr.setMotorCmd('FR_2',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 1, Kd = 1, tau = torque))
    mCmdArr.setMotorCmd('FL_2',  motorCmd(mode=MotorModeLow.Servo, q=pos, dq = vel, Kp = 1, Kd = 1, tau = torque))

    lcmd.motorCmd = mCmdArr
    cmd_bytes = lcmd.buildCmd(debug=False)
    conn.send(cmd_bytes)
    

flag = True
count = 0    
while True:
    if flag == True :
        count += 1
        if count >= 120 :
            flag = False
            count = 0
        print("Moving to - deg" +count.__str__())
        move(-math.radians(-30),0.3)
    if flag == False :
        count += 1
        if count >= 120 :
            flag = True
            count = 0
        move(-math.radians(240),0.3)
        print("Moving to + deg" +count.__str__())
    time.sleep(1/60) #command at 60Hz
    

Moving to - deg1
Moving to - deg2
Moving to - deg3
Moving to - deg4
Moving to - deg5
Moving to - deg6
Moving to - deg7
Moving to - deg8
Moving to - deg9
Moving to - deg10
Moving to - deg11


Moving to - deg12
Moving to - deg13
Moving to - deg14
Moving to - deg15
Moving to - deg16
Moving to - deg17
Moving to - deg18
Moving to - deg19
Moving to - deg20
Moving to - deg21
Moving to - deg22
Moving to - deg23
Moving to - deg24
Moving to - deg25
Moving to - deg26
Moving to - deg27
Moving to - deg28
Moving to - deg29
Moving to - deg30
Moving to - deg31
Moving to - deg32
Moving to - deg33
Moving to - deg34
Moving to - deg35
Moving to - deg36
Moving to - deg37
Moving to - deg38
Moving to - deg39
Moving to - deg40
Moving to - deg41
Moving to - deg42
Moving to - deg43
Moving to - deg44
Moving to - deg45
Moving to - deg46
Moving to - deg47
Moving to - deg48
Moving to - deg49
Moving to - deg50
Moving to - deg51
Moving to - deg52
Moving to - deg53
Moving to - deg54
Moving to - deg55
Moving to - deg56
Moving to - deg57
Moving to - deg58
Moving to - deg59
Moving to - deg60
Moving to - deg61
Moving to - deg62
Moving to - deg63
Moving to - deg64
Moving to - deg65
Moving to - deg66
Moving to 

KeyboardInterrupt: 

In [303]:
    
segment = 0
flag = True
while True:
    if flag == True :
        segment += 1
        if segment >= 800 :
            flag = False
            segment = 0
        # moveFL2(d2r(-240), 0.1) # Backward Limit
        moveFL2(0, segment/1000) # Backward Limit
        print("spd", segment/1000)
        time.sleep(0.01)
    if flag == False :
        segment += 1
        if segment >= 500 :
            flag = True
            segment = 0
        moveFL2(0, -segment/1000) # Backward Limit
        print("spd", -segment/1000)
        time.sleep(0.01)

spd 0.001
spd 0.002
spd 0.003
spd 0.004
spd 0.005
spd 0.006
spd 0.007
spd 0.008
spd 0.009
spd 0.01
spd 0.011
spd 0.012
spd 0.013
spd 0.014
spd 0.015
spd 0.016
spd 0.017
spd 0.018
spd 0.019
spd 0.02
spd 0.021
spd 0.022
spd 0.023
spd 0.024
spd 0.025
spd 0.026
spd 0.027
spd 0.028
spd 0.029
spd 0.03
spd 0.031
spd 0.032
spd 0.033
spd 0.034
spd 0.035
spd 0.036
spd 0.037
spd 0.038
spd 0.039
spd 0.04
spd 0.041
spd 0.042
spd 0.043
spd 0.044
spd 0.045
spd 0.046
spd 0.047
spd 0.048
spd 0.049
spd 0.05
spd 0.051
spd 0.052
spd 0.053
spd 0.054
spd 0.055
spd 0.056
spd 0.057
spd 0.058
spd 0.059
spd 0.06
spd 0.061
spd 0.062
spd 0.063
spd 0.064
spd 0.065
spd 0.066
spd 0.067
spd 0.068
spd 0.069
spd 0.07
spd 0.071
spd 0.072
spd 0.073
spd 0.074
spd 0.075
spd 0.076
spd 0.077
spd 0.078
spd 0.079
spd 0.08
spd 0.081
spd 0.082
spd 0.083
spd 0.084
spd 0.085
spd 0.086
spd 0.087
spd 0.088
spd 0.089
spd 0.09
spd 0.091
spd 0.092
spd 0.093
spd 0.094
spd 0.095
spd 0.096
spd 0.097
spd 0.098
spd 0.099
spd 0.1
spd 0.101
s

KeyboardInterrupt: 

In [15]:
jointnum = 3
minmaxjoint = [[0,0],[0,0],[0,0]] # [[min1,max1],[min2,max2],[min3,max3]]
diffjoint = [0] * jointnum
for i in range(jointnum):
    diff = minmaxjoint[i][1] - minmaxjoint[i][0]
    print(diff)
    diffjoint[i] = diff

0
0
0


In [13]:
i = 0
minmaxjoint[i][1] - minmaxjoint[i][0]

0

In [28]:
def printJointStatus(conn, lstate, motor_keys):
    # 데이터를 가져옵니다.
    data = conn.getData()

    # 데이터 패킷을 반복하여 처리합니다.
    paket = data[-1]
    lstate.parseData(paket)

    # 리스트로 받은 motor_keys에 대해 처리합니다.
    for key in motor_keys:
        # motorState에 해당 key가 존재한다고 가정하고 q 값을 출력합니다.
        if key in lstate.motorState:
            motor_state = lstate.motorState[key]
            print(f'MotorState {key} q: {round(math.degrees(motor_state.q),1)}')

# 사용 예시
motor_keys = ['FL_2', 'FR_2', 'RL_2', 'RR_2']  # 예시로 여러 키 입력
while True:
    print("test")
    printJointStatus(conn, lstate, motor_keys)
    time.sleep(0.02)  # 0.05초 간격으로 출력


test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


KeyboardInterrupt: 

In [26]:
motiontime = 0
while True:
    time.sleep(0.002)
    motiontime +=1
    data = conn.getData()
    for paket in data:
        lstate.parseData(paket)
        if motiontime % 100 == 0: #Print every 100 cycles
            print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
            print(f'MotorState FL_2 q:\t\t{math.degrees(lstate.motorState[d["FL_2"]].q)}')
            print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')


+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
MotorState FL_2 q:		-164.5861668448268
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+

KeyboardInterrupt: 